# Create a local information system of a Thin-Section

### TEST files

requirements:

- Python 3
- GDAL

## 1. Input and settings

In this section we collect input data and global settings.
We assume that we have a raster file of a Thin-section digitalized in any way and we have defined a polygon feature that overlay minerals or other elements of the thin-section.

The poligon feature layer must be in a GIS-readable format.
We can't use a custom CRS (probaly non geospatial), because GDAL command do not work.

TODO: modify GDAL scripts or make a custom script version of `gdal2tiles.py` able to work with non-geospatial and custom CRS.

In [1]:
# verify Python version
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
else :
    print("Python 3 detected. OK.")

# verify GDAL version
try:
    from osgeo import gdal
    print(f"GDAL version: {gdal.__version__}")
except ImportError:
    raise ImportError("GDAL not detected.")

Python 3 detected. OK.
GDAL version: 3.8.5


In [2]:
from lis_functions import upload_files_widgets
upload_files_widgets()

Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile polygon layer', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Select the EPSG code of files (not work with custom projection)')

Text(value='', description='EPSG:', placeholder='EPSG code')

Label(value='Note: ouptut folder will be created in the same directory as the input files')

In [3]:
# Exectute the following code to generate tiles from a raster file

from lis_functions import save_to_temp_dir
save_to_temp_dir()


#Save the uploaded files to the temporary directory
import shutil



# # Import subprocess module
# import subprocess

# # Raster file path from which to generate tiles
# input_raster = 'path/to/your/input.tif'

# # Directory di output per i tile
# output_directory = 'path/to/your/output_tiles'

# # Comando gdal2tiles.py
# gdal2tiles_command = [
#     'gdal2tiles.py',
#     '-p', 'mercator',
#     '-z', '0-5',
#     input_raster,
#     output_directory
# ]
# # Esegui il comando
# try:
#     subprocess.run(gdal2tiles_command, check=True)
#     print(f'Tile generati con successo nella directory: {output_directory}')
# except subprocess.CalledProcessError as e:
#     print(f'Errore durante la generazione dei tile: {e}')

Errore durante la generazione dei tile: Command '['gdal2tiles.py', '-p', 'mercator', '-z', '0-5', 'path/to/your/input.tif', 'path/to/your/output_tiles']' returned non-zero exit status 2.


Usage: gdal2tiles.py [options] input_file [output]

gdal2tiles.py: error: The provided input file path/to/your/input.tif does not exist or is not a file
